# Simulation of RCT EMS v2 - Weekdays

Want to graph:
 - Travel Times
 - Coverages

In [1]:
%matplotlib notebook

import pandas
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from sys import argv
import scipy.stats as stats
from scipy.interpolate import make_interp_spline, BSpline
import numpy as np
from numpy import median, mean
from functools import reduce
from collections import Counter
from os import listdir

import subprocess


# Execute the Simulator
## Print the results

In [2]:
commands = [
    'python3 run.py configurations/exp-scenarios/tj.yaml'
]

for c in commands:
    print(c)
    p = subprocess.Popen(c.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    p.communicate()
    print("Done.")
    

python3 run.py configurations/exp-scenarios/tj.yaml
Done.


In [3]:
results = 'results/tijuana-multi-scenario/'
csvs = []
for f in listdir(results):
    
    path = results + f
    print(path)
    p = pandas.read_csv(path)
    csvs.append(p)


FileNotFoundError: [Errno 2] No such file or directory: 'results/tijuana-multi-scenario/'

In [ ]:
# csvs[0]

In [ ]:
# csvs[1]

In [ ]:
# csvs[2]

In [ ]:
# csvs[3]

In [ ]:
# csvs[4]

# Want to see number of events per hour.

In [ ]:
cases = csvs[1]
dates = [datetime.strptime(arr[1], '%Y-%m-%d %H:%M:%S.%f').replace(
minute=0, second=0, microsecond=0) for arr in cases.values]


In [ ]:
# matplotlib does not play well with datetimes

first = dates[0]
deltas = [(date-first).total_seconds()/3600 for date in dates]


In [ ]:
c = Counter(deltas)
sum(c.values())

In [ ]:
l = list(deltas)

plt.figure("Histogram: The number of cases per hour")
ys, xs, patches = plt.hist(deltas, bins=int((max(deltas)+1)/1), histtype='step')

plt.xlabel("Number of hours since starting"); plt.ylabel('Number of cases')
plt.ylim(0,10)

plt.show()
ys = np.append(ys, [0])

plt.figure("Bar graph: The number of cases per hour")
plt.bar(c.keys(), c.values())

plt.xlabel("Number of hours since starting"); plt.ylabel('Number of cases')
plt.ylim(0,10)

plt.show()


In [ ]:
# Smoothed the original
smooth_xs = np.linspace(int(min(c.keys())), int(max(c.keys())), len(c.keys()))

spl = make_interp_spline(list(c.keys()), list(c.values()), k=3)
smooth_ys = spl(smooth_xs)

# plt.figure("This is the curve you were looking for.")
# plt.plot(smooth_xs, smooth_ys,)
# plt.show()

plt.figure("overlay")
plt.hist(deltas, bins=int((max(deltas)+1)), histtype='step', color='black')
plt.xlabel("Number of hours since starting"); plt.ylabel('Number of cases')
plt.ylim(0,10)
plt.plot(smooth_xs, smooth_ys, '-', color='red')

plt.show()


In [ ]:
# I want to know the number of times each count per hour occurred. 

hours_counted = Counter(list(c.values()))

plt.figure('Number of times each number of cases per hour occurred')
plt.bar(hours_counted.keys(), hours_counted.values())

plt.xlabel('Number of cases within a single hour')
plt.ylabel('Count number of cases occured in 1 hour')

plt.show()

# Want to see the number of cases that have been reached within r1 and r2

In [ ]:
# to incident durations
durations = [pandas.to_timedelta(array[7]) for array in  cases.values]
r1 = timedelta(minutes=10)
r2 = timedelta(minutes=14)

In [ ]:
times = {
    'lt_r1' : list(filter(lambda t: t <  r1, durations)),
    'lt_r2' : list(filter(lambda t: t <  r2 and t >= r1, durations)),
    'beyond': list(filter(lambda t: t >= r2, durations)),
}

counts = {k:len(v) for k,v in times.items()}
counts

In [ ]:
plt.figure("Travel times for the TO_INCIDENT events ")

plt.bar(counts.keys(), counts.values())
plt.xlabel("Event TO_INCIDENT times separated by travel time"); plt.ylabel('Number of cases')
plt.show()

# Want to plot the histogram for the coverage

In [ ]:
metrics = csvs[2]
coverages = [(pandas.to_datetime(arr[0]), float(arr[1])) for arr in metrics.values]

xs = [cov[0] for cov in coverages]
ys = [cov[1] for cov in coverages]

# list(map(print, xs))
# list(map(print, ys))

first = xs[0]
xs = [(date-first).total_seconds()/3600 for date in xs]

# Smoothed the original
smooth_xs = np.linspace(int(min(xs)), int(max(xs)), len(xs)//4)

spl = make_interp_spline(xs, ys, k=1)
smooth_ys = spl(smooth_xs)


plt.figure("Coverages over time ")
plt.ylim(-0.1, 1.0)
plt.xlabel("Number of hours since starting"); plt.ylabel('Coverage')

plt.plot(xs, ys, 'b.' )
plt.plot(smooth_xs, smooth_ys, '-', color='red')

plt.show()